In [ ]:
!pip install -q stellargraph

     |████████████████████████████████| 435 kB 4.3 MB/s 
     |████████████████████████████████| 462 kB 70.9 MB/s 


In [ ]:
import csv
import time 
import math
import random
import numpy as np
import pandas as pd
import networkx as nx
import tensorflow as tf
import stellargraph as sg
from stellargraph import StellarGraph
from stellargraph.layer import DeepGraphCNN,GAT
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification

from sklearn.datasets import load_files
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D,Conv2D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/GECCO-2022/KPnoties.zip"


In [ ]:
def extractKPparams(file_path):
  objects = []
  lines = []

  start = 2
  with open(file_path) as f:
    lines = f.read().splitlines()

  capacity = lines[start][0]
  for i in range(start+1, len(lines)-1):
    line = lines[i].split( )
    objects.append([int(line[0]),int(line[1])])
  #print(instance)
  return capacity, objects

In [ ]:
import os
knap_frame = pd.read_csv('/content/knapsackNoTies.csv')
instanceNames = knap_frame.Instance
KPinstances = []
KPlabels = []

for i,name in enumerate(instanceNames):
  file_path = os.path.join("/content/knapsackNoTies", name)
  #print(knap_frame.iloc[i].Instance + "   " + name)
  if not os.path.isfile(file_path):
    print(name)
  elif knap_frame.iloc[i].Label ==1:
    KP_capacity, KP_objects = extractKPparams(file_path)
    KPinstance = (name, KP_capacity, KP_objects)
     
    KPinstances.append(KPinstance)
    KPlabels.append(0)
  elif knap_frame.iloc[i].Label ==2:
    KP_capacity, KP_objects = extractKPparams(file_path)
    KPinstance = (name, KP_capacity, KP_objects)
     
    KPinstances.append(KPinstance)
    KPlabels.append(1)

KP_labels = pd.DataFrame(KPlabels)


In [ ]:
from sklearn.preprocessing import Normalizer
KPinstances_norm = []

for inst in KPinstances:
  a2 = [[i / sum(j) for i in j] for j in inst[2]]
  KPinstances_norm.append((inst[0],inst[1],a2))
#print(a2)
#NS = np.array(NormalizeSet)
#transformer = Normalizer().fit(NS) 
#transformer.transform(NS)

In [ ]:
instancesKP = []
#instancesKPNX = []
counter = 0

for instance in KPinstances_norm:
  tempG = nx.Graph(name = instance[0])
  tempG.add_node(0, profitweight=[0,math.log(float(instance[1]))])
  for count, vals in enumerate(instance[2]):
    tempG.add_node(count+1, profitweight=[vals[0], vals[1]])
    tempG.add_edge(0, count+1)

  #instancesKPNX.append(tempG)
  SG = StellarGraph.from_networkx(tempG, node_features="profitweight")
  instancesKP.append(SG)
  if(len(instance[1])==0):
    print("empty")
    print(counter)
  counter = counter +1
tempG.nodes[1]
generator = PaddedGraphGenerator(graphs=instancesKP)
#print(SG)

In [ ]:
#from stellargraph import datasets
#dataset = datasets.MUTAG()
#graphs, graph_labels = dataset.load()
#GClabels = pd.get_dummies(graph_labels, drop_first=True)
#generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
def DGCNN(generator):
  model = DeepGraphCNN(
                 layer_sizes=[32, 32, 32, 1],
                 activations=["tanh","tanh", "tanh", "tanh"],
                 generator=generator,
                 k=32
    )
  x_inp, x_out = model.in_out_tensors()

  x_out = Conv1D(filters=16, kernel_size=64, strides=64)(x_out)
  x_out = MaxPool1D(pool_size=1)(x_out)
  x_out = Conv1D(filters=32, kernel_size=8, strides=1)(x_out)
  x_out = Flatten()(x_out)
  x_out = Dense(units=258, activation="relu")(x_out)
  x_out = Dropout(rate=0.5)(x_out)
  predictions = Dense(units=1, activation="sigmoid")(x_out)

  model = Model(inputs=x_inp, outputs=predictions)
  return model

def GCN(generator):
    gc_model = GCNSupervisedGraphClassification(
        layer_sizes=[32, 32],
        activations=["relu", "relu"],
        generator=generator,
        dropout=0.5,
    )
    x_inp, x_out = gc_model.in_out_tensors()
    predictions = Dense(units=32, activation="relu")(x_out)
    predictions = Dense(units=16, activation="relu")(predictions)
    predictions = Dense(units=1, activation="sigmoid")(predictions)

    # Let's create the Keras model and prepare it for training
    model = Model(inputs=x_inp, outputs=predictions)
    return model

def GATmodel(generator):
  gat = GAT(
    layer_sizes=[64, 64],
    activations=["elu", "softmax"],
    attn_heads=8,
    generator=generator,
    in_dropout=0.5,
    attn_dropout=0.5,
    normalize=None,
  )
  x_inp, x_out = gat.in_out_tensors()
  predictions = Dense(units=32, activation="relu")(x_out)
  predictions = Dense(units=16, activation="relu")(predictions)
  predictions = Dense(units=1, activation="sigmoid")(predictions)

  # Let's create the Keras model and prepare it for training
  model = Model(inputs=x_inp, outputs=predictions)
  return model



In [ ]:
def train_fold(model, train_gen, test_gen, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=1
    )
    # calculate performance on the test data and return along with history
    test_metrics = model.evaluate(test_gen)
    test_acc = test_metrics
    #print(test_metrics)
    #print(test_gen)
    predicted_y = model.predict(test_gen)
    #print(predicted_y)
    return (history, test_acc, test_gen.targets,predicted_y)

In [ ]:
epochs = 20  # maximum number of training epochs
folds = 2  # the number of folds for k-fold cross validation
n_repeats = 1  # the number of repeats for repeated k-fold cross validation

In [ ]:
def get_GC_generators(GC_train_index, GC_test_index, graph_labels, batch_size):
    GC_train_gen = generator.flow(
        GC_train_index, targets=graph_labels.iloc[GC_train_index].values, batch_size=batch_size
    )
    GC_test_gen = generator.flow(
        GC_test_index, targets=graph_labels.iloc[GC_test_index].values, batch_size=batch_size
    )

    return GC_train_gen, GC_test_gen

In [ ]:
test_accs2 = []
alltests = []
allpredictions = []
#stratified_folds2 = model_selection.RepeatedStratifiedKFold(
#    n_splits=folds, n_repeats=n_repeats
#).split(KP_labels, KP_labels)
cv = KFold(n_splits=folds, random_state=1, shuffle=True)
#for i, (train_index2, test_index2) in enumerate(stratified_folds2):
for train_index, test_index in cv.split(KP_labels, KP_labels):
    print(f"Training and evaluating on fold {1+1} out of {folds * n_repeats}...")
    train_gen2, test_gen2 = get_GC_generators(
        train_index, test_index, KP_labels, batch_size=1
    )

    gnnmodel = GCN(generator)
    gnnmodel.compile(optimizer=Adam(learning_rate=0.001), loss=binary_crossentropy, metrics=["acc"])
    history = train_fold(gnnmodel, train_gen2, test_gen2, epochs)
    alltests.append(history[2])
    allpredictions.append(history[3])
    test_accs2.append(history[1])

In [ ]:
predictions = []
for sublist in allpredictions:
    for item in sublist:
      if item >= 0.5:
        predictions.append(1)
      else:
        predictions.append(0)

tests = []
for sublist in alltests:
    for item2 in sublist:
        tests.append(item2)

In [ ]:
print(predictions)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 

In [ ]:
allrepo=classification_report(tests, predictions)
print(allrepo)

In [ ]:
df = pd.DataFrame(allrepo).transpose()
df.to_latex('G-GraphColur100.tex')


In [ ]:
sg.utils.plot_history(history)


AttributeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=13)